
Описание данных:

В данной лабораторной работе представлены данные по диабету среди коренных жительниц США. В данном датасете представлены следующие признаки:


1.   Количестов беременностей
2.   Глюкоза
3.   Кровяное давление
4.   Толщина кожи
5.   Инсулин
6.   BMI
7.   Diabetes Pedigree Function
8.   Возраст
9.   Исход

Все признаки, кроме исхода, являются вещественными, а исход является бинарным признаком, где 1 - больной, а 0 - здоровый пациент.

Выполнение работы:

Сначала подключаемся к облачному хранилищу для работы с файлом и подключаем все необходимые библиотеки.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

import numpy as np
import pandas as pd

DiabetSet = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/diabetes (1).csv")
DiabetSet.head()

Mounted at /content/gdrive


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Изменим тип исхода на категориальную переменную.

In [ ]:
DiabetSet = DiabetSet.astype({'Outcome': 'category'})

Далее с помощью методы dtype определяем типы данных, которые представлены в данной таблице. В данном случае типы данных это int64, float64 и category

In [ ]:
DiabetSet.dtypes

Pregnancies                    int64
Glucose                        int64
BloodPressure                  int64
SkinThickness                  int64
Insulin                        int64
BMI                          float64
DiabetesPedigreeFunction     float64
Age                            int64
Outcome                     category
dtype: object

Для более коректного сравнения произведем замену нулевых значений на среднее, на основании предыдущей группировки по исходу, данная замена позволит более четко проследить зависимость и впоследствии построить какую-либо модель.

In [ ]:
Indexes = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for line in range(len(DiabetSet)):
  for col in range(1, len(DiabetSet.loc[line])-1):
    if DiabetSet.loc[line, Indexes[col]] == 0:
      DiabetSet.loc[line, Indexes[col]] = DiabetSet.groupby(['Outcome']).mean().loc[DiabetSet.loc[line, 'Outcome'], Indexes[col]]
DiabetSet.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.000000,100.335821,33.6,0.627,50,1
1,1,85.0,66.0,29.000000,68.792000,26.6,0.351,31,0
2,8,183.0,64.0,22.164179,100.710208,23.3,0.672,32,1
3,1,89.0,66.0,23.000000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.000000,168.000000,43.1,2.288,33,1


Первой библиотекой, которой мы  воспользуемся будет H20. Для этого установим ее и подключимся к ней.

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
h2o.init()

In [ ]:
DiabetSet.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

Выделим признаки, которые являются независимыми, а также результирующую переменную.

In [ ]:
predictors = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
response = "Outcome"

Конвертируем датафрейм Pandas в H2Oframe

In [ ]:
from h2o.estimators.extended_isolation_forest import H2OFrame
data = H2OFrame(DiabetSet)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Далее необходимо изменить тип переменной "Исход" на факторный, для корректной работы программы.

In [ ]:
data['Outcome'] = data['Outcome'].asfactor()

Проведем разделение выборки на тестовую и валидационную

In [ ]:
train, valid = data.split_frame(ratios = [.8], seed = 1234)

In [ ]:
diabet_gbm = H2OGradientBoostingEstimator(sample_rate = .7, seed = 1234)
diabet_gbm.train(x = predictors,
                   y = response,
                   training_frame = train,
                   validation_frame = valid)

perf = diabet_gbm.model_performance(valid)
perf

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.10199334663172917
RMSE: 0.3193639720314882
LogLoss: 0.3392277042256054
Mean Per-Class Error: 0.13631221719457015
AUC: 0.914027149321267
AUCPR: 0.8570770395300517
Gini: 0.8280542986425341

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4123356622053312
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      94   10   0.0962   (10.0/104.0)
1      9    42   0.1765   (9.0/51.0)
Total  103  52   0.1226   (19.0/155.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.412336     0.815534  51
max f2                       0.126253     0.841924  86
max f0point5                 0.696878     0.862069  37
max accuracy                 0.696878     0.877419  37
max precision                0.977423     1         0
max recall                   0.0167663    1         139
max specificity              0.977423     1         0
max absolute_mcc             0.412336     0.723836  51
max min_per_class_accuracy   0.367574     0.843137  57
max mean_per_class_accuracy  0.412336     0.863688  51
max tns                      0.977423     104       0
max fns                      0.977423     50        0
max fps                      0.0084402    104       154
max tps                      0.0167663    51        139
max tnr                      0.977423     1         0
max fnr                      0.977423     0.980392  0
max fpr                      0.0084402    1         154
max tpr                      0.0167663    1         139

Gains/Lift Table: Avg response rate: 32.90 %, avg score: 33.47 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0129032                   0.973603           3.03922   3.03922            1                0.976426   1                           0.976426            0.0392157       0.0392157                  203.922   203.922            0.0392157
2        0.0258065                   0.965955           3.03922   3.03922            1                0.96935    1                           0.972888            0.0392157       0.0784314                  203.922   203.922            0.0784314
3        0.0322581                   0.956815           3.03922   3.03922            1                0.957938   1                           0.969898            0.0196078       0.0980392                  203.922   203.922            0.0980392
4        0.0451613                   0.953231           3.03922   3.03922            1                0.954905   1                           0.965614            0.0392157       0.137255                   203.922   203.922            0.137255
5        0.0516129                   0.949789           3.03922   3.03922            1                0.95086    1                           0.96377             0.0196078       0.156863                   203.922   203.922            0.156863
6        0.103226                    0.932933           2.27941   2.65931            0.75             0.941967   0.875                       0.952869            0.117647        0.27451                    127.941   165.931            0.255279
7        0.154839                    0.857954           2.65931   2.65931            0.875            0.900582   0.875                       0.93544             0.137255        0.411765                   165.931   165.931            0.382919
8        0.2                         0.779085          

По результатм данной библиотеки удалось получить точность в 87,7%, f-мера 81,1%. Данные показатели ниже, чем те, которые были получены при обычном машинном обучение:

RfOver Accuracy: 0.9 Precision: 0.87 Recall 0.93 F1: 0.9 ROC_AUC: 0.9

В целом ухудшение не сильное, но ручное обучения дает лучший результат.

Далее попробуем воспользоваться библиотекой TPOT и посмотрим на ее результаты. Аналогично устанавливаем библиотеку и подключаемся к ней

In [ ]:
!pip install deap update_checker tqdm stopit xgboost

In [ ]:
!pip install tpot

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

Разделим датасет на X и y, а также разобьем выборку на тестовую и тренировочную.

In [ ]:
X = DiabetSet.drop(('Outcome'), axis=1)
y = DiabetSet['Outcome']
feature_names = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
from tpot import TPOTClassifier 

tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8472113943028485

Generation 2 - Current best internal CV score: 0.8489505247376311

Generation 3 - Current best internal CV score: 0.8489505247376311

Generation 4 - Current best internal CV score: 0.8489505247376311

Generation 5 - Current best internal CV score: 0.8489505247376311

Best pipeline: BernoulliNB(GaussianNB(XGBClassifier(input_matrix, learning_rate=0.5, max_depth=3, min_child_weight=8, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0)), alpha=10.0, fit_prior=False)
0.796875


Таким образом данная библиотека дала на результат в районе 84,8%, что хуже чем при применении предыдущей библиотеки, а также ручного машинного обучения.

